In [1]:
import pandas as pd
from pandasql import sqldf 
import datetime # cambio de formato de fecha
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cycler
import numpy as np
import seaborn as sns

pysqldf = lambda q: sqldf(q, globals())

df = pd.read_csv("facebook_df.csv", sep=";")

In [2]:
df.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


Podemos observar que los nombres de las columnas no un tanto confusos y podemos sacar palabras clave para renombrarlas e determinar mucho mejor su uso. 
En específico, las columnas con nombre de lifetime podemos sustituir con el prefijo 'lf' o acompañarlo de la primera letra de la palabra compún; como 't' para 'total' y 'p' para post'; y una palabra clave. 
* Lifetime Post Total Reach - ltpt Reach
* Lifetime Post Total Impressions - ltpt Impressions
* Lifetime Engaged Users - lt Engaged Users
* Lifetime Post Consumers - ltp Consumers
* Lifetime Post Compumptions - ltp Consumptions
* Lifetime Post Impressions by people who have liked your Page - ltpi liked page 
* Lifetime Post reach by people who like your Page - ltp reach pel page
* Lifetime People who have liked your Page and engaged with your post - lfpe liked and engaged post 

In [3]:
# Cambiamos el nombre a las columnas
df = df.rename(columns = { 'Page total likes':'TotalLikes',
                            'Post Month':'Month',
                            'Post Weekday':'Weekday',
                            'Post Hour':'Hour',
                            'Lifetime Post Total Reach':'Reach', 
                            'Lifetime Post Total Impressions':'Impressions', 
                            'Lifetime Engaged Users':'EngagedUsers',
                            'Lifetime Post Consumers':'Consumers',
                            'Lifetime Post Consumptions': 'Consumptions',
                            'Lifetime Post Impressions by people who have liked your Page':'Followers',
                            'Lifetime Post reach by people who like your Page':'LikeFollowers',
                            'Lifetime People who have liked your Page and engaged with your post':'EngagedFollowers',
                            'Total Interactions':'TotalInteract'})
df.head()

,TotalLikes,Type,Category,Month,Weekday,Hour,Paid,Reach,Impressions,EngagedUsers,Consumers,Consumptions,Followers,LikeFollowers,EngagedFollowers,comment,like,share,TotalInteract
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TotalLikes        500 non-null    int64  
 1   Type              500 non-null    object 
 2   Category          500 non-null    int64  
 3   Month             500 non-null    int64  
 4   Weekday           500 non-null    int64  
 5   Hour              500 non-null    int64  
 6   Paid              499 non-null    float64
 7   Reach             500 non-null    int64  
 8   Impressions       500 non-null    int64  
 9   EngagedUsers      500 non-null    int64  
 10  Consumers         500 non-null    int64  
 11  Consumptions      500 non-null    int64  
 12  Followers         500 non-null    int64  
 13  LikeFollowers     500 non-null    int64  
 14  EngagedFollowers  500 non-null    int64  
 15  comment           500 non-null    int64  
 16  like              499 non-null    float64
 1

In [5]:
# Aquí obtenemos específicamente qúe renglones tienen valores faltantes, para identificar si son datos relevantes para el cliente. 
nan_rows = df[df.isnull().any(1)]
nan_rows

/tmp/ipykernel_19090/2546253675.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  nan_rows = df[df.isnull().any(1)]


,TotalLikes,Type,Category,Month,Weekday,Hour,Paid,Reach,Impressions,EngagedUsers,Consumers,Consumptions,Followers,LikeFollowers,EngagedFollowers,comment,like,share,TotalInteract
111,136736,Photo,1,10,6,8,0.0,1261,2158,37,37,49,1911,1077,33,0,NaN,NaN,0
120,136393,Photo,1,10,7,9,0.0,584,1029,273,271,308,943,511,232,0,2.0,NaN,2
124,136393,Photo,1,10,7,6,0.0,677,1285,251,246,297,1210,615,211,0,7.0,NaN,7
164,135428,Photo,1,9,5,10,0.0,1060,2004,266,251,337,1705,870,204,0,18.0,NaN,18
499,81370,Photo,2,1,4,4,NaN,4188,7292,564,524,743,3861,2200,316,0,91.0,28.0,119


In [6]:
# Remplazaremos los nans con '0' para poder utilizar la columna más versatilmente 
df[df['Paid'].isnull()]
df['Paid'] = df['Paid'].fillna(0)

df[df['like'].isnull()]
df['like'] = df['like'].fillna(0)
df['like'] = df['like'].astype(int)

df[df['share'].isnull()]
df['share'] = df['share'].fillna(0)
df['share'] = df['share'].astype(int)

df['Hour'] = df['Hour'].replace(to_replace = 0, value = 24)
# ¿Cambiar nombre de categorías?
# 1 == action
# 2 == product
# 3 == inspiration


In [7]:
# Revisamos que se haya cambiado correctamente
df.loc[111:'like'].head(1)

,TotalLikes,Type,Category,Month,Weekday,Hour,Paid,Reach,Impressions,EngagedUsers,Consumers,Consumptions,Followers,LikeFollowers,EngagedFollowers,comment,like,share,TotalInteract
111,136736,Photo,1,10,6,8,0.0,1261,2158,37,37,49,1911,1077,33,0,0,0,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TotalLikes        500 non-null    int64  
 1   Type              500 non-null    object 
 2   Category          500 non-null    int64  
 3   Month             500 non-null    int64  
 4   Weekday           500 non-null    int64  
 5   Hour              500 non-null    int64  
 6   Paid              500 non-null    float64
 7   Reach             500 non-null    int64  
 8   Impressions       500 non-null    int64  
 9   EngagedUsers      500 non-null    int64  
 10  Consumers         500 non-null    int64  
 11  Consumptions      500 non-null    int64  
 12  Followers         500 non-null    int64  
 13  LikeFollowers     500 non-null    int64  
 14  EngagedFollowers  500 non-null    int64  
 15  comment           500 non-null    int64  
 16  like              500 non-null    int64  
 1

In [9]:
df['TotalLikes'].max()

139441

In [10]:
df['TotalLikes'].min()

81370

In [11]:
#saving csv
df['Category'] = df['Category'].astype(str)
df['Category'] = df['Category'].replace('1', 'action')
df['Category'] = df['Category'].replace('2', 'product')
df['Category'] = df['Category'].replace('3', 'inspiration')
df.to_csv("clean.csv")

Ya que vemos que no hay espacios en blanco podemos iniciar cierto análisis. 
Empezaremos eligiendo las variables de mayor importancia para el cliente.
* Que tiempos presentan mayor impacto (mes, día, hora)
    * Mes nos ayuda a identificar tendencias por TEMPORADA durante el año. 
    * Día nos ayuda a determinar comportamiento a lo largo de la semana. 
    * Hora sirve para identificar momentos de mayor tráfico en la página por día. 
* Que tipos de interacciones se encuentran (likes, compartir, comentarios, interacciones totales)
    * Likes son una muestra clara de respuesta/interés a un post. 
    * Comentarios demuestran interés mayor a solo observar el post, puede fomentar aún más interacción.
    * Compartir nos indica un interés del consumidor en extender el alcance del post. 
* Quienes interactuan con los posts (usuarios generales, seguidores de la página) y cómo. 
    * Usuarios generales pueden ayudar a identificar TENDENCIAS ATRACTIVAS para el público común. 
    * Seguidores de la página nos muestran QUÉ LOS MANTIENE INTERESADOS. 

In [12]:
df['comment'].max()

372

In [13]:
# Cantidad de interacciones por Mes
tendf = pysqldf("""
SELECT Month, like, comment, share, TotalInteract
from df
""")
tendf.head()

,Month,like,comment,share,TotalInteract
0,12,79,4,17,100
1,12,130,5,29,164
2,12,66,0,14,80
3,12,1572,58,147,1777
4,12,325,19,49,393


In [14]:
# Aquí podemos graficar e identificar posibles tendencias por temporada (primavera, verano, etc)
tendf = tendf.groupby('Month').sum()
tendf

,like,comment,share,TotalInteract
Month,,,,
1,3072,149,794,4015
2,5017,274,1002,6293
3,2992,104,398,3494
4,9137,266,1473,10876
5,7998,293,1192,9483
6,6547,317,864,7728
7,14171,797,2114,17082
8,6747,194,722,7663
9,8314,397,1315,10026


In [15]:
# También podemos determinar por día y hora
daydf = pysqldf("""
SELECT Weekday, like, comment, share, TotalInteract
FROM df
""")
daydf = daydf.groupby('Weekday').sum()
daydf

,like,comment,share,TotalInteract
Weekday,,,,
1,13895,539,1684,16118
2,11036,431,1763,13230
3,15145,870,2402,18417
4,15750,735,2273,18758
5,11534,408,1814,13756
6,10875,429,1883,13187
7,10560,329,1705,12594


In [16]:
hourdf = pysqldf("""
SELECT Hour, like, comment, share, TotalInteract
FROM df
""")
hourdf.info() # Confirmamos primero que no se estén perdiendo datos, sí hay 500s y no hay pérdida de datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Hour           500 non-null    int64
 1   like           500 non-null    int64
 2   comment        500 non-null    int64
 3   share          500 non-null    int64
 4   TotalInteract  500 non-null    int64
dtypes: int64(5)
memory usage: 19.7 KB


In [17]:
hourdf = hourdf.groupby('Hour').sum()
hourdf

,like,comment,share,TotalInteract
Hour,,,,
1,586,26,112,724
2,6071,314,1068,7453
3,20227,656,3119,24002
4,4815,237,831,5883
5,7409,417,1070,8896
6,2124,53,337,2514
7,1663,30,237,1930
8,874,35,174,1083
9,3405,104,484,3993


In [18]:
# Otra  métrica para identificar tendencia: ¿Qué tipo de post contiene el mayor tipo de presencia y cómo? 
presdf = pysqldf("""
SELECT Type, like, comment, share, TotalInteract
FROM df
""")
presdf = presdf.groupby('Type').sum()
presdf

,like,comment,share,TotalInteract
Type,,,,
Link,1613,62,284,1959
Photo,77610,3192,11461,92263
Status,7952,401,1414,9767
Video,1620,86,365,2071


In [19]:
# Tendencias posibles para usuarios que siguen la página 

# Interacciones dependiendo de tipos de post
foldf = pysqldf("""
SELECT Type, Followers as ImpressionsFollowers, LikeFollowers as ReachFollowers, 
    EngagedFollowers, Impressions, EngagedUsers, Reach
FROM df
""")
foldf = foldf.groupby('Type').sum()
foldf

,ImpressionsFollowers,ReachFollowers,EngagedFollowers,Impressions,EngagedUsers,Reach
Type,,,,,,
Link,245269,143976,4632,631960,7542,407981
Photo,6995978,2581178,216112,12351656,348871,5596709
Status,839892,445886,77393,1091001,91810,588550
Video,302049,121704,6856,718357,11949,358440


In [20]:
# Interacciones por categoría
usrcatdf = pysqldf("""
SELECT Category, Followers as ImpressionsFollowers, LikeFollowers as ReachFollowers, 
    EngagedFollowers, Impressions, EngagedUsers, Reach
FROM df
""")
usrcatdf = usrcatdf.groupby('Category').sum()
usrcatdf

,ImpressionsFollowers,ReachFollowers,EngagedFollowers,Impressions,EngagedUsers,Reach
Category,,,,,,
action,4561250,1620584,113317,8685000,182992,3939011
inspiration,2339139,884032,82341,3826027,129193,1730134
product,1482799,788128,109335,2281947,147987,1282535


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TotalLikes        500 non-null    int64  
 1   Type              500 non-null    object 
 2   Category          500 non-null    object 
 3   Month             500 non-null    int64  
 4   Weekday           500 non-null    int64  
 5   Hour              500 non-null    int64  
 6   Paid              500 non-null    float64
 7   Reach             500 non-null    int64  
 8   Impressions       500 non-null    int64  
 9   EngagedUsers      500 non-null    int64  
 10  Consumers         500 non-null    int64  
 11  Consumptions      500 non-null    int64  
 12  Followers         500 non-null    int64  
 13  LikeFollowers     500 non-null    int64  
 14  EngagedFollowers  500 non-null    int64  
 15  comment           500 non-null    int64  
 16  like              500 non-null    int64  
 1

Aplicamos un nuevo grupo, donde observaremos las tendencias en relación a los seguidores
* Followers representa las 'Impresiones por parte de gente que le ha dado like a la página', osea que el post ha APARECIDO en su pantalla. 
* ReachFollowers representa los registros de personas que vieron el post PORQUE SIGUEN LA PÁGINA. 
* EngagedFollowers representa los registros de las INTERACCIONES por parte de followers en el post. 

In [22]:
# 
likemonth = pysqldf("""
SELECT Month, Followers, LikeFollowers as ReachFollowers, EngagedFollowers
FROM df
""")
likemonth = likemonth.groupby('Month').sum()
likemonth

,Followers,ReachFollowers,EngagedFollowers
Month,,,
1,235976,135444,11984
2,1158292,227958,18883
3,1619746,233150,16791
4,745237,417592,31516
5,426339,243374,22006
6,634185,332009,29433
7,815869,434163,38220
8,640849,278027,21768
9,341762,176227,25479


In [23]:
# En los casos de día y hora
likeday = pysqldf("""
SELECT Weekday, Followers, LikeFollowers as ReachFollowers, EngagedFollowers
FROM df
""")
likeday = likeday.groupby('Weekday').sum()
likeday

,Followers,ReachFollowers,EngagedFollowers
Weekday,,,
1,1383613,438020,41280
2,1050961,488943,34237
3,939535,476520,48117
4,2212978,547066,41696
5,793810,399874,42019
6,1164738,499561,48061
7,837553,442760,49583


In [24]:
likehour = pysqldf("""
SELECT Hour, Followers, LikeFollowers as ReachFollowers, EngagedFollowers
FROM df
""")
likehour = likehour.groupby('Hour').sum()
likehour

,Followers,ReachFollowers,EngagedFollowers
Hour,,,
1,15818,8501,1524
2,455593,235661,24055
3,1297743,630365,58794
4,417261,218031,21651
5,235344,125058,9295
6,202541,114877,8309
7,1218115,113638,7718
8,43794,23910,3582
9,250179,132847,17594
